## **Importing Libraries** 

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['GLOG_minloglevel'] = '2'
import gc
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from concurrent.futures import ThreadPoolExecutor
from IPython.display import HTML
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import tensorflow as tf

2024-06-12 19:05:00.171819: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 19:05:00.171863: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 19:05:00.173168: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
np.random.seed(42)
tf.random.set_seed(42)

---

## **Landmark Detection**

In [3]:
HAND_FILTERS = list(range(21))
POSE_FILTERS = [0, 1, 2, 3, 4, 5, 6, 11, 12, 13, 14, 15, 16]
FACE_FILTERS = [
    61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
    291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
    95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
]
HAND_FILTERS_LEN = len(HAND_FILTERS) # 21
POSE_FILTERS_LEN = len(POSE_FILTERS) # 13
FACE_FILTERS_LEN = len(FACE_FILTERS) # 40
TOTAL_FILTERS_LEN = HAND_FILTERS_LEN * 2 + POSE_FILTERS_LEN + FACE_FILTERS_LEN

In [4]:
vision_running_mode = vision.RunningMode
base_options = python.BaseOptions

hands_options_image = vision.HandLandmarkerOptions(
    running_mode=vision_running_mode.IMAGE,
    min_hand_detection_confidence=0.55,
    base_options=base_options(model_asset_path='datasets/mediapipe_models/hand_landmarker.task'),
    num_hands=2
)

pose_options_image = vision.PoseLandmarkerOptions(
    running_mode=vision_running_mode.IMAGE,
    min_pose_detection_confidence=0.55,
    base_options=base_options(model_asset_path='datasets/mediapipe_models/pose_landmarker_full.task')
)

face_options_image = vision.FaceLandmarkerOptions(
    running_mode=vision_running_mode.IMAGE,
    min_face_detection_confidence=0.55,
    base_options=base_options(model_asset_path='datasets/mediapipe_models/face_landmarker.task')
)

hands_options_video = vision.HandLandmarkerOptions(
    running_mode=vision_running_mode.VIDEO,
    min_hand_detection_confidence=0.55,
    base_options=base_options(model_asset_path='datasets/mediapipe_models/hand_landmarker.task'),
    num_hands=2
)

pose_options_video = vision.PoseLandmarkerOptions(
    running_mode=vision_running_mode.VIDEO,
    min_pose_detection_confidence=0.55,
    base_options=base_options(model_asset_path='datasets/mediapipe_models/pose_landmarker_full.task')
)

face_options_video = vision.FaceLandmarkerOptions(
    running_mode=vision_running_mode.VIDEO,
    min_face_detection_confidence=0.55,
    base_options=base_options(model_asset_path='datasets/mediapipe_models/face_landmarker.task')
)

hands_detector_image = vision.HandLandmarker.create_from_options(hands_options_image)
pose_detector_image = vision.PoseLandmarker.create_from_options(pose_options_image)
face_detector_image = vision.FaceLandmarker.create_from_options(face_options_image)
IMAGE_DETECTORS = (hands_detector_image, pose_detector_image, face_detector_image)

I0000 00:00:1718193902.881860     516 task_runner.cc:85] GPU suport is not available: INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:77) display != EGL_NO_DISPLAYeglGetDisplay() returned error 0x300c
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1718193902.918102     695 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718193902.930957     695 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1718193902.931596     516 task_runner.cc:85] GPU suport is not available: INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:77) display != EGL_NO_DISPLAYeglGetDisplay() returned error 0x300c
W0000 00:00:1718193902.989898     719 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabl

In [5]:
def detect(detector, image, frame_timestamp=None):
    if frame_timestamp is None:
        return detector.detect(image)
    else:
        return detector.detect_for_video(image, frame_timestamp)

def process_landmarks(landmarks, filters, start_idx, landmarks_array):
    for i in filters:
        landmarks_array[start_idx] = [landmarks[i].x, landmarks[i].y]
        start_idx += 1
    return start_idx

def extract_landmarks_from_image(image, detectors, timestamp=None):
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img_rgb)
    
    hands_detector, pose_detector, face_detector = detectors

    with ThreadPoolExecutor() as executor:
        detect_args = (mp_image, timestamp) if timestamp != None else (mp_image,)
        
        hands_future = executor.submit(detect, hands_detector, *detect_args)
        pose_future = executor.submit(detect, pose_detector, *detect_args)
        face_future = executor.submit(detect, face_detector, *detect_args)
        
        hand_result = hands_future.result()
        pose_result = pose_future.result()
        face_result = face_future.result()

    hand_landmarks = hand_result.hand_landmarks
    pose_landmarks = pose_result.pose_landmarks
    face_landmarks = face_result.face_landmarks

    landmarks_array = np.full((TOTAL_FILTERS_LEN, 2), np.nan)
    arr_idx = 0
    
    if hand_landmarks:
        if hand_result.handedness[0].index == 1:
            arr_idx += HAND_FILTERS_LEN
        
        for landmarks in hand_landmarks:
            arr_idx = process_landmarks(landmarks, HAND_FILTERS, arr_idx, landmarks_array)
        
        if arr_idx == HAND_FILTERS_LEN:
            arr_idx += HAND_FILTERS_LEN
    else:
        arr_idx += HAND_FILTERS_LEN*2

    if pose_landmarks:
        arr_idx = process_landmarks(pose_landmarks[0], POSE_FILTERS, arr_idx, landmarks_array)
    else:
        arr_idx += POSE_FILTERS_LEN

    if face_landmarks:
        arr_idx = process_landmarks(face_landmarks[0], FACE_FILTERS, arr_idx, landmarks_array)
    else:
        arr_idx += FACE_FILTERS_LEN
    
    return landmarks_array

def extract_landmarks_from_video(video_path, start_frame=1, end_frame=-1):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = round(cap.get(cv2.CAP_PROP_FPS))
    frame_duration_ms = 1000 / fps
    
    hands_detector_video = vision.HandLandmarker.create_from_options(hands_options_video)
    pose_detector_video = vision.PoseLandmarker.create_from_options(pose_options_video)
    face_detector_video = vision.FaceLandmarker.create_from_options(face_options_video)
    VIDEO_DETECTORS = (hands_detector_video, pose_detector_video, face_detector_video)
    
    if start_frame < 1:
        start_frame = 1
    elif start_frame > total_frames:
        start_frame = 1
    
    if end_frame < 0 or end_frame > total_frames:
        end_frame = total_frames
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame - 1)
    video_landmarks = np.zeros((end_frame - start_frame + 1, TOTAL_FILTERS_LEN, 2), dtype=object)
    
    for frame_idx in range(start_frame, end_frame + 1):
        ret, frame = cap.read()
        if not ret:
            break
        
        timestamp = int((frame_idx - 1) * frame_duration_ms)
        
        # landmarks = extract_landmarks_from_image(frame, IMAGE_DETECTORS)
        landmarks = extract_landmarks_from_image(frame, VIDEO_DETECTORS, timestamp)
        video_landmarks[frame_idx - start_frame] = landmarks
    
    cap.release()
    return video_landmarks

In [6]:
def draw_landmarks_image(image, landmarks, dot_size=5):
    for landmark in landmarks:
        x, y = landmark
        if not np.isnan(x) and not np.isnan(y):
            cv2.circle(image, (int(x * image.shape[1]), int(y * image.shape[0])), dot_size, (0, 255, 0), -1)
    return image

def draw_landmarks_video(input_path, output_path, video_landmarks, start_frame=1, end_frame=-1, dot_size=5):
    cap = cv2.VideoCapture(input_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'avc1'), fps, (frame_width, frame_height))

    if end_frame == -1 or end_frame > total_frames:
        end_frame = total_frames

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame - 1)

    while cap.isOpened():
        frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        if frame_number > end_frame:
            break

        ret, frame = cap.read()
        if not ret:
            break

        frame_landmarks = video_landmarks[frame_number - 1]
        annotated_frame = draw_landmarks_image(frame, frame_landmarks, dot_size=dot_size)

        out.write(annotated_frame)

    cap.release()
    out.release()

In [7]:
def plot_frame(frame, ax):
    ax.clear()
    x = frame[:, 0]
    y = frame[:, 1]
    ax.scatter(x, y, color='dodgerblue')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.invert_yaxis()
    ax.set_xticklabels([])
    ax.set_yticklabels([])

def animate_frames(data):
    fig, ax = plt.subplots()
    anim = FuncAnimation(fig, lambda frame: plot_frame(data[frame], ax), frames=range(data.shape[0]), interval=100)
    plt.close(fig)
    return HTML(anim.to_jshtml())

---

## **Augmentation**

In [8]:
def padding(X, length=None, pad=0):
    if length is None:
        length = X.shape[0]
    
    if X.shape[0] > length:
        X_padded = X[:length]
    else:
        pad_length = length - X.shape[0]
        X_padded = np.pad(
            X, ((0, pad_length), (0, 0), (0, 0)),
            mode='constant', constant_values=pad
        )
            
    return X_padded

def remove_no_hands(video):
    frames_to_keep = []
    for i, frame in enumerate(video):
        hand_landmarks_data = frame[:43]
        if not np.all(np.isnan(hand_landmarks_data)):
            frames_to_keep.append(i)
    video_with_hands = video[frames_to_keep]
    return video_with_hands

def is_dominant_hand(video):

    left_hand_sum = np.sum(~np.isnan(video[:, slice(0, 21)]), axis=1)
    right_hand_sum = np.sum(~np.isnan(video[:, slice(21, 43)]), axis=1)

    left_dominant_count = np.sum(left_hand_sum >= right_hand_sum)
    right_dominant_count = np.sum(left_hand_sum < right_hand_sum)

    return left_dominant_count > right_dominant_count

def hflip(data):
    data[:, :, 0] = 1 - data[:, :, 0]
    return data

In [9]:
MAX_FRAME_LENGTH = 64

In [10]:
def predict_preprocess(video):
    if not is_dominant_hand(video):
        hflip(video)
    video = remove_no_hands(video)
    np.nan_to_num(video, copy=False, nan=0)
    video = padding(video, MAX_FRAME_LENGTH, -100)
    return video

## **Modelling**

In [11]:
class EarlyLateDropout(tf.keras.layers.Layer):
    def __init__(self, early_rate, late_rate, switch_epoch, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.early_rate = early_rate
        self.late_rate = late_rate
        self.switch_epoch = switch_epoch
        self.dropout = tf.keras.layers.Dropout(early_rate)
    
    def build(self, input_shape):
        super().build(input_shape)
        agg = tf.VariableAggregation.ONLY_FIRST_REPLICA
        self._train_counter = self.add_weight(name="train_counter", shape=[], dtype=tf.int64, aggregation=agg, trainable=False)

    def call(self, inputs, training=False):
        if training:
            dropout_rate = tf.cond(self._train_counter < self.switch_epoch, lambda: self.early_rate, lambda: self.late_rate)
            x = self.dropout(inputs, training=training)
            x = tf.keras.layers.Dropout(dropout_rate)(x, training=training)
            self._train_counter.assign_add(1)
        else:
            x = inputs
        return x

In [12]:
def add_dummy_channel(x, fill_value=0):
    dummy_channel_shape = tf.concat([tf.shape(x)[:-1], [1]], axis=0)
    dummy_channel = tf.fill(dummy_channel_shape, tf.cast(fill_value, x.dtype))
    result = tf.concat([x, dummy_channel], axis=-1)
    return result

In [13]:
def scce_with_ls(y_true, y_pred):
    y_true = tf.cast(y_true, tf.int32)
    y_true = tf.one_hot(y_true, NUM_CLASSES, axis=1)
    y_true = tf.squeeze(y_true, axis=2)
    return tf.keras.losses.categorical_crossentropy(y_true, y_pred, label_smoothing=0.1)

In [15]:
model = tf.keras.models.load_model('a16.h5', custom_objects={'EarlyLateDropout': EarlyLateDropout, 'scce_with_ls': scce_with_ls, 'add_dummy_channel': add_dummy_channel})

In [40]:
video_path = 'working/real_test/airplane.mp4'
output_path = 'working/real_test/percobaan_output_video.mp4'
video = extract_landmarks_from_video(video_path).astype(np.float32)
draw_landmarks_video(video_path, output_path, video, dot_size=5)
animate_frames(video)

I0000 00:00:1718193999.526069     516 task_runner.cc:85] GPU suport is not available: INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:77) display != EGL_NO_DISPLAYeglGetDisplay() returned error 0x300c
W0000 00:00:1718193999.533509    2623 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718193999.540650    2637 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1718193999.541445     516 task_runner.cc:85] GPU suport is not available: INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:77) display != EGL_NO_DISPLAYeglGetDisplay() returned error 0x300c
W0000 00:00:1718193999.582006    2646 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718193999

In [41]:
gc.collect()

8779

In [42]:
video_preprocessed = predict_preprocess(video.copy())
animate_frames(video_preprocessed)

In [43]:
predictions = model.predict(np.expand_dims(video_preprocessed, axis=0))

1/1 [==============================] - 0s 16ms/step


In [44]:
def load_decoder(json_file_path):
    with open(json_file_path, 'r') as json_file:
        label_to_int = json.load(json_file)
    
    int_to_label = {v: k for k, v in label_to_int.items()}
    return int_to_label

decoder_dict = load_decoder('label_encoder.json')

In [45]:
def decode_predictions(predictions, decoder_dict, num_predictions=10):
    top_prediction = np.argmax(predictions)
    top_confidence = predictions[top_prediction] * 100
    top_label = decoder_dict[top_prediction]

    top_indices = np.argsort(predictions)[-num_predictions:][::-1]
    top_confidences = predictions[top_indices] * 100
    top_labels = [decoder_dict[i] for i in top_indices]

    top_1_prediction = top_labels[0]
    top_1_confidence = top_confidences[0]

    top_1_prediction = (top_labels[0], top_confidences[0])
    top_n_predictions = [(label, confidence) for label, confidence in zip(top_labels, top_confidences)]

    return top_1_prediction, top_n_predictions

In [46]:
top_1, top_n = decode_predictions(predictions[0], decoder_dict, num_predictions=201)
top_1

('airplane', 52.188427)

In [47]:
for pred in top_n:
    print(pred[0]+',', pred[1])

airplane, 52.188427
moon, 0.45095804
bath, 0.43290448
loud, 0.4069725
cow, 0.3848548
up, 0.35353258
child, 0.34413674
hide, 0.34310073
bye, 0.33844906
stairs, 0.33204487
not, 0.33153406
there, 0.33084032
uncle, 0.32772326
close, 0.3258658
many, 0.32482
blue, 0.3242261
every, 0.32396805
drink, 0.32366416
empty, 0.32222128
better, 0.31825915
wet, 0.31391525
sleepy, 0.31259274
why, 0.30836836
dance, 0.30833822
ear, 0.30681422
grandma, 0.30525458
touch, 0.30432978
bedroom, 0.3042272
boat, 0.3041176
hat, 0.30293295
please, 0.30166948
gift, 0.2995309
farm, 0.29709053
no, 0.29582703
penny, 0.29564187
where, 0.29337925
high, 0.29202694
cat, 0.29171395
dry, 0.29170963
wolf, 0.28994757
outside, 0.28879705
before, 0.28613088
black, 0.28504768
hungry, 0.2844017
fine, 0.28281972
car, 0.28234878
bee, 0.27886817
fish, 0.2776675
milk, 0.2763118
bad, 0.276257
yellow, 0.27496213
later, 0.27451083
same, 0.2704011
yesterday, 0.26814666
police, 0.2679247
chocolate, 0.26580545
bird, 0.2657436
mouse, 0.26484